# GeekBrains University
## Methods of data collection

## Lesson 3
### MongoDB and SQLite Database Management Systems in Python

### The Task 
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать
функцию, записывающую собранные вакансии в созданную БД.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной
платой больше введённой суммы.
3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с
сайта.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import itertools

#### Fetch the number of pages (see the homework Lesson 2)

In [2]:
def pages_counter_hh(main_link,  occupation= 'санитар', headers= {}):
    
    #request for the first page
    response = requests.get(f'{main_link}/search/vacancy?area=1&st=searchVacancy&text={occupation}', headers=headers).text
    soup = bs(response,'lxml')
    
    # Look for the number of pages on the footer
    pages = soup.find_all('a', {'class' : "bloko-button HH-Pager-Control"})
    pages = int(pages[-1].text)
    
    return(pages)

In [3]:
def pages_counter_sj(main_link,  occupation= 'санитар', headers= {}):
    
    #request for the first page
    response = requests.get(f'{main_link}/vacancy/search/?keywords={occupation}', headers=headers).text
    soup = bs(response,'lxml')
    
    # Look for the number of pages on the footer
    pages = soup.find('div', {'class' : "L1p51"}).find_all('span', {'class' : "_3IDf-"})
    pages = int(pages[-2].text)
    
    return(pages)

#### Parcers for every vacancy (see the homework Lesson 2)

In [4]:
def parser_hh(vacancy, main_link):
    v_data = {}    
    
    title = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
    if not title:
        v_data['title'] = np.nan
    else:
        v_data['title'] = title.text
    
    link = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-title'})
    if not link:
        v_data['link'] = np.nan
    else:
        v_data['link'] = link['href']
        
    employer = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-employer'})
    if not employer:
        v_data['employer'] = np.nan
    else:
        v_data['employer'] = employer.text
    
    compensation = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
    if not compensation:
        v_data['compensation'] = np.nan
    else:
        v_data['compensation'] = compensation.text
    
    v_data['source'] = main_link
    
    return(v_data)

In [5]:
def parser_sj(vacancy, main_link):
    
    v_data = {}
    
    title = vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
    if not title:
        v_data['title'] = np.nan
    else:
        title = title.find('a')
        if not title:
            v_data['title'] = np.nan
        else:
            v_data['title'] = title.text
    
    link = vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
    if not link:
        v_data['link'] = np.nan
    else:
        link = link.find('a')
        if not link:
            v_data['link'] = np.nan
        else:
            v_data['link'] = main_link + link['href']
    
    employer = vacancy.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'})
    if not employer:
        v_data['employer'] = np.nan
    else:
        employer = employer.find('a')
        if not employer:
            v_data['employer'] = np.nan
        else:
            v_data['employer'] = employer.text
    
    compensation = vacancy.find('span', {'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
    if not compensation:
        v_data['compensation'] = np.nan
    else:
        v_data['compensation'] = compensation.text
        
    v_data['source'] = main_link
    
    return(v_data)

#### Parcer for compensation string (min, max, currency) (see the homework Lesson 2)

In [6]:
def compensation_parser(v_data):
    v_data['compensation_min'] = np.nan
    v_data['compensation_max'] = np.nan
    v_data['compensation_currency'] = np.nan
    
    if v_data['compensation'] == np.nan:
        return(v_data)
    
    compensation_tt = str(v_data['compensation'])
    compensation_tt = compensation_tt.replace(u'\xa0', ' ')
    
    numbers = '[\d+\s]*\d+' # pattern for number values
    
    fr = re.search(f'^от {numbers}', compensation_tt) # 'от 30 000'
    if fr:
        v_data['compensation_min'] = float(fr[0][3:].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '')
        
    fr = re.search(f'^{numbers}[\s]*[\-—]+', compensation_tt) # '30 000-'
    if fr:
        v_data['compensation_min'] = float(fr[0][:-1].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '-')
    
    fr = re.search(f'^до {numbers}', compensation_tt) # 'до 30 000'
    if fr:
        v_data['compensation_max'] = float(fr[0][3:].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '')
        
        
    fr = re.search(f'^[\-—]+[\s]*{numbers}', compensation_tt) # '-30 000'
    if fr:
        v_data['compensation_max'] = float(fr[0][1:].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '')
        
    fr = re.search(f'^[\s]*{numbers}', compensation_tt) # '30 000'
    if fr:
        v_data['compensation_max'] = float(fr[0].replace(' ', ''))
        v_data['compensation_min'] = float(fr[0].replace(' ', ''))
        compensation_tt = compensation_tt.replace(fr[0], '')
    
    v_data['compensation_currency'] = compensation_tt # the rest to the currency
    
    return(v_data)

---

#### Function for comparing the new vacancy data with data of vacancies, existing in the base

In [7]:
def vacancy_comparing(collection, v_data):
    
    # Return '0' for existing document and the existing document data
    if collection.count_documents(v_data):
        return(0, collection.find(v_data)[0]) 
    
    #Return '-1' for a changed document: the 'new' vacancy has the same link as the one in the base
    elif collection.count_documents({'link' : v_data['link']}):
        return(-1, collection.find({'link' : v_data['link']})[0])
    
    # Return '1' for the vacancy with a new link
    else:
        return(1, {})

#### Function insert completely new and update (if it's neccessary) partially new vacancies for given site (links)

In [8]:
def db_insert_update(collection, pages, m_link, vacancy_pattern, main_link, headers,
                     update, the_first_page_only, parser):
    
    counter = np.zeros(2) # Let the function return the number of completely new and updated vacancies
    
    n_pages = (pages, 1)[the_first_page_only]
    
    for page in range(n_pages):
        
        # Get the soup part for every vacancy
        link = m_link + f'{page}'
        response = requests.get(link, headers=headers).text
        soup = bs(response,'lxml')
        vacancies = soup.find_all('div', vacancy_pattern)
        
        for vacancy in vacancies:
            # Parse the required field
            v_data = parser(vacancy, main_link)
            v_data = compensation_parser(v_data)

            comparing_result = vacancy_comparing(collection, v_data)
            
            if comparing_result[0] == 0: # No action for existing data
                continue
                
            elif comparing_result[0] == 1: # Insert completely new vacancy
                collection.insert_one(v_data)
                counter[0] += 1
                
            elif update: # Update changed vacancy if parametr 'update' is True 
                collection.update_one(comparing_result[1], {'$set' : v_data})
                counter[1] += 1

    return counter

#### Function aggregate the job of 'db_insert_update' function for both sites 

In [9]:
def db_insert_update_two_site(collection, occupation= 'санитар', headers= {}, update= True, the_first_page_only= False):
    
    counter = np.zeros((2, 2)) # Let the function return the number of completely new and updated vacancies
    
    # The HeadHunter part:
    main_link = 'https://hh.ru'
    pages = pages_counter_hh(main_link,  occupation, headers)
    link = f'{main_link}/search/vacancy?area=1&st=searchVacancy&text={occupation}&page='
    vacancy_pattern = {'data-qa':'vacancy-serp__vacancy'}
    parser = parser_hh
           
    counter[0] += db_insert_update(collection, pages, link, vacancy_pattern,
                                   main_link, headers, update, the_first_page_only,
                                  parser)
    
    #---------------------------------------------------------------------------------------------------------
    # The SuperJob part:
    main_link = 'https://www.superjob.ru'
    pages = pages_counter_sj(main_link,  occupation, headers)
    link = f'{main_link}/vacancy/search/?keywords={occupation}&page='
    vacancy_pattern = {'class':'_3zucV f-test-vacancy-item _3j3cA RwN9e _3tNK- _1NStQ _1I1pc'}
    parser = parser_sj
    
    counter[1] += db_insert_update(collection, pages, link, vacancy_pattern, 
                                   main_link, headers, update, the_first_page_only,
                                  parser)
    
    return counter

---

### The main solution

#### The first way: Using the exising database, created on the previous lesson:

In [10]:
client = MongoClient( 'localhost' , 27017 )
db = client['vacancies']
collection = db.vacancies_1

In [11]:
df = pd.read_csv('hh_sj_completed.csv', header= 0, index_col= 0)
df.head(3)

,compensation,employer,link,source,title,compensation_min,compensation_max,compensation_currency
0,от 40 000 руб.,Амориа,https://hh.ru/vacancy/36577813?query=%D1%81%D0...,https://hh.ru,Санитар/санитарка,40000.0,NaN,руб.
1,от 28 000 руб.,АО Столичные аптеки,https://hh.ru/vacancy/36381593?query=%D1%81%D0...,https://hh.ru,Санитарка/Санитар в аптеку (м. Раменки),28000.0,NaN,руб.
2,34 000-34 000 руб.,ООО Нейро-Клиника,https://hh.ru/vacancy/36472038?query=%D1%81%D0...,https://hh.ru,Санитарка/санитар,34000.0,34000.0,руб.


In [12]:
df_dict = df.to_dict(orient= 'index')
dict(itertools.islice(df_dict.items(), 3))

{0: {'compensation': 'от 40\xa0000 руб.',
  'employer': ' Амориа',
  'link': 'https://hh.ru/vacancy/36577813?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80',
  'source': 'https://hh.ru',
  'title': 'Санитар/санитарка',
  'compensation_min': 40000.0,
  'compensation_max': nan,
  'compensation_currency': ' руб.'},
 1: {'compensation': 'от 28\xa0000 руб.',
  'employer': 'АО Столичные аптеки',
  'link': 'https://hh.ru/vacancy/36381593?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80',
  'source': 'https://hh.ru',
  'title': 'Санитарка/Санитар в аптеку (м. Раменки)',
  'compensation_min': 28000.0,
  'compensation_max': nan,
  'compensation_currency': ' руб.'},
 2: {'compensation': '34\xa0000-34\xa0000 руб.',
  'employer': 'ООО Нейро-Клиника',
  'link': 'https://hh.ru/vacancy/36472038?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80',
  'source': 'https://hh.ru',
  'title': 'Санитарка/санитар',
  'compensation_min': 34000.0,
  'compensation_max': 34000.0,
  'compensation_currency': ' руб.'

In [13]:
df_list = [*df_dict.values()]
df_list[:3]

[{'compensation': 'от 40\xa0000 руб.',
  'employer': ' Амориа',
  'link': 'https://hh.ru/vacancy/36577813?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80',
  'source': 'https://hh.ru',
  'title': 'Санитар/санитарка',
  'compensation_min': 40000.0,
  'compensation_max': nan,
  'compensation_currency': ' руб.'},
 {'compensation': 'от 28\xa0000 руб.',
  'employer': 'АО Столичные аптеки',
  'link': 'https://hh.ru/vacancy/36381593?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80',
  'source': 'https://hh.ru',
  'title': 'Санитарка/Санитар в аптеку (м. Раменки)',
  'compensation_min': 28000.0,
  'compensation_max': nan,
  'compensation_currency': ' руб.'},
 {'compensation': '34\xa0000-34\xa0000 руб.',
  'employer': 'ООО Нейро-Клиника',
  'link': 'https://hh.ru/vacancy/36472038?query=%D1%81%D0%B0%D0%BD%D0%B8%D1%82%D0%B0%D1%80',
  'source': 'https://hh.ru',
  'title': 'Санитарка/санитар',
  'compensation_min': 34000.0,
  'compensation_max': 34000.0,
  'compensation_currency': ' руб.'}]

In [14]:
collection.insert_many(df_list)

#### Slicing

In [15]:
required_compensation = 30000 
result = collection.find({'compensation_min':{'$gt' : required_compensation}}).sort('compensation_min', -1)
df = pd.DataFrame.from_dict(result)

df

,_id,compensation,compensation_currency,compensation_max,compensation_min,employer,link,source,title
0,5e960eaa5cbc29c51b3234e8,от 400 000 руб.,руб.,NaN,400000.0,RRC Consulting,https://hh.ru/vacancy/36158068?query=%D1%81%D0...,https://hh.ru,Директор МТО
1,5e98c1f8d94d85391d3a7720,от 400 000 руб.,руб.,NaN,400000.0,RRC Consulting,https://hh.ru/vacancy/36158068?query=%D1%81%D0...,https://hh.ru,Директор МТО
2,5e960eaa5cbc29c51b323514,250 000-250 000 руб.,руб.,250000.0,250000.0,NaN,https://hh.ru/vacancy/36289748?query=%D1%81%D0...,https://hh.ru,"Главный инженер (Водопровод, канализация, дожд..."
3,5e98c1f8d94d85391d3a774c,250 000-250 000 руб.,руб.,250000.0,250000.0,NaN,https://hh.ru/vacancy/36289748?query=%D1%81%D0...,https://hh.ru,"Главный инженер (Водопровод, канализация, дожд..."
4,5e960eaa5cbc29c51b32341e,от 200 000 руб.,руб.,NaN,200000.0,ООО Центрпроект,https://hh.ru/vacancy/36394467?query=%D1%81%D0...,https://hh.ru,Главный инженер по строительству
5,5e98c1f8d94d85391d3a7656,от 200 000 руб.,руб.,NaN,200000.0,ООО Центрпроект,https://hh.ru/vacancy/36394467?query=%D1%81%D0...,https://hh.ru,Главный инженер по строительству
6,5e960eaa5cbc29c51b3233ed,от 150 000 руб.,руб.,NaN,150000.0,ООО Стимул Групп,https://hh.ru/vacancy/36587033?query=%D1%81%D0...,https://hh.ru,Прораб
7,5e960eaa5cbc29c51b323476,150 000-200 000 руб.,руб.,200000.0,150000.0,ООО Коннектом,https://hh.ru/vacancy/36573269?query=%D1%81%D0...,https://hh.ru,Разработчик Python / CV
8,5e960eaa5cbc29c51b32348f,от 150 000 руб.,руб.,NaN,150000.0,ООО БраерСтрой,https://hh.ru/vacancy/36283907?query=%D1%81%D0...,https://hh.ru,Главный инженер
9,5e960eaa5cbc29c51b3234ae,от 150 000 руб.,руб.,NaN,150000.0,ООО Группа компаний АРКС,https://hh.ru/vacancy/36500999?query=%D1%81%D0...,https://hh.ru,Начальник участка/строительство мостов


#### The second way: creating the new base from the scratch

In [16]:
collection = db.vacancies_3 # It's neccessary to comment this line to proceed working with the existing base

In [17]:
collection.drop({})  # It's neccessary to comment this line to proceed working with the existing base

In [18]:
headers = {'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
occupation = 'санитар'

#### Create the base partially for the first pages only (to check insert and update later)

In [19]:
def print_results(counter, total_instances):
    print('HH:')
    print('---------------------------------------------------------------------------------\n')
    print(f' Completely new instances {counter[0][0]} \n Updated instances {counter[0][1]}')
    print('---------------------------------------------------------------------------------\n')
    print('SUperJob:')
    print('---------------------------------------------------------------------------------\n')
    print(f' Completely new instances {counter[1][0]} \n Updated instances {counter[1][1]}')
    print('---------------------------------------------------------------------------------\n')
    print(f'Total number of instances {total_instances}')

In [20]:
update= True
the_first_page_only= True
counter = db_insert_update_two_site(collection, occupation, headers, update, the_first_page_only)
total_instances = collection.count_documents({})
print_results(counter, total_instances)

HH:
---------------------------------------------------------------------------------

 Completely new instances 50.0 
 Updated instances 0.0
---------------------------------------------------------------------------------

SUperJob:
---------------------------------------------------------------------------------

 Completely new instances 20.0 
 Updated instances 0.0
---------------------------------------------------------------------------------

Total number of instances 70


#### Insert and update

In [21]:
update= True
the_first_page_only= False
counter = db_insert_update_two_site(collection, occupation, headers, update, the_first_page_only)
total_instances = collection.count_documents({})
print_results(counter, total_instances)

HH:
---------------------------------------------------------------------------------

 Completely new instances 355.0 
 Updated instances 0.0
---------------------------------------------------------------------------------

SUperJob:
---------------------------------------------------------------------------------

 Completely new instances 38.0 
 Updated instances 0.0
---------------------------------------------------------------------------------

Total number of instances 463
